In [24]:
import pandas as pd
import requests
from datetime import datetime
from dotenv import load_dotenv
import os
import import_ipynb



# Making the API call and viewing the json

If you want to search for the city by name and country instead of latitude and longitude, try using the [Geocoding API](https://openweathermap.org/api/geocoding-api).

In [25]:
# Berlin
latitude = 52.520008
longitude = 13.404954

load_dotenv()
API_key = os.getenv('weather_api_key')  # Make sure this matches the name exactly in the .env file

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")

response = requests.get(url)
json = response.json()

json

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1730905200,
   'main': {'temp': 8.06,
    'feels_like': 8.06,
    'temp_min': 8.06,
    'temp_max': 9.39,
    'pressure': 1031,
    'sea_level': 1031,
    'grnd_level': 1025,
    'humidity': 77,
    'temp_kf': -1.33},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 100},
   'wind': {'speed': 0.39, 'deg': 109, 'gust': 0.38},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2024-11-06 15:00:00'},
  {'dt': 1730916000,
   'main': {'temp': 8.06,
    'feels_like': 8.06,
    'temp_min': 8.05,
    'temp_max': 8.06,
    'pressure': 1031,
    'sea_level': 1031,
    'grnd_level': 1026,
    'humidity': 75,
    'temp_kf': 0.01},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}],
   'clouds': {'all': 99},
   'wind': {'speed': 0.4, 'deg': 342, 'gust': 0.42},
   'visibility': 

# Exploring the json
Remember, if you want to know more about any of the information in the json, or you're unsure what a key is, check out the [open weather docs](https://openweathermap.org/forecast5).

First, let's look at the keys

In [26]:
json.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [27]:
json["cod"]

'200'

In [28]:
json["message"]

0

In [29]:
json["cnt"]

40

In [30]:
json["list"]

[{'dt': 1730905200,
  'main': {'temp': 8.06,
   'feels_like': 8.06,
   'temp_min': 8.06,
   'temp_max': 9.39,
   'pressure': 1031,
   'sea_level': 1031,
   'grnd_level': 1025,
   'humidity': 77,
   'temp_kf': -1.33},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}],
  'clouds': {'all': 100},
  'wind': {'speed': 0.39, 'deg': 109, 'gust': 0.38},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2024-11-06 15:00:00'},
 {'dt': 1730916000,
  'main': {'temp': 8.06,
   'feels_like': 8.06,
   'temp_min': 8.05,
   'temp_max': 8.06,
   'pressure': 1031,
   'sea_level': 1031,
   'grnd_level': 1026,
   'humidity': 75,
   'temp_kf': 0.01},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04n'}],
  'clouds': {'all': 99},
  'wind': {'speed': 0.4, 'deg': 342, 'gust': 0.42},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-11-06 18:00:00'},
 {'dt': 1

Looks like the information we'll want is in `list`. Let's look at the first element of `list` to investigate further.

In [31]:
json["list"][0]

{'dt': 1730905200,
 'main': {'temp': 8.06,
  'feels_like': 8.06,
  'temp_min': 8.06,
  'temp_max': 9.39,
  'pressure': 1031,
  'sea_level': 1031,
  'grnd_level': 1025,
  'humidity': 77,
  'temp_kf': -1.33},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'clouds': {'all': 100},
 'wind': {'speed': 0.39, 'deg': 109, 'gust': 0.38},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-11-06 15:00:00'}

In [32]:
json["city"]

{'id': 6545310,
 'name': 'Mitte',
 'coord': {'lat': 52.52, 'lon': 13.405},
 'country': 'DE',
 'population': 329078,
 'timezone': 3600,
 'sunrise': 1730873541,
 'sunset': 1730906862}

# Building a DataFrame

## What would we like from the json?
Let's begin by exploring the first 2 items of `list` and see what we have.

In [33]:
json["list"][0]

{'dt': 1730905200,
 'main': {'temp': 8.06,
  'feels_like': 8.06,
  'temp_min': 8.06,
  'temp_max': 9.39,
  'pressure': 1031,
  'sea_level': 1031,
  'grnd_level': 1025,
  'humidity': 77,
  'temp_kf': -1.33},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'clouds': {'all': 100},
 'wind': {'speed': 0.39, 'deg': 109, 'gust': 0.38},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-11-06 15:00:00'}

In [34]:
json["list"][0]

{'dt': 1730905200,
 'main': {'temp': 8.06,
  'feels_like': 8.06,
  'temp_min': 8.06,
  'temp_max': 9.39,
  'pressure': 1031,
  'sea_level': 1031,
  'grnd_level': 1025,
  'humidity': 77,
  'temp_kf': -1.33},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'clouds': {'all': 100},
 'wind': {'speed': 0.39, 'deg': 109, 'gust': 0.38},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-11-06 15:00:00'}

When deciding what is useful, remember the project goals.

The time the forecast is for will be useful.

In [35]:
json["list"][0]["dt_txt"]

'2024-11-06 15:00:00'

The temperature will be useful. Scooters don't have heaters!

In [13]:
json["list"][0]["main"]["temp"]

8.06

The forecast will be useful. Scooters don't have a roof!

In [36]:
json["list"][0]["weather"][0]["main"]

'Clouds'

If it's been recently raining will be useful. Wet conditions will affect scooter rentals.

In [37]:
json["list"][0]["rain"]["3h"]

KeyError: 'rain'

In [38]:
json["list"][0].get("rain",{}).get("3h",0)

0

> If you get an error on 'rain', it's because it hasn't rained in the last 3 hours. But we know this key exists, because we read the [docs](https://openweathermap.org/forecast5)

Wind speed will be useful. It's likely to affect whether people rent scooters or not.

In [39]:
json["list"][0]["wind"]["speed"]

0.39

In [40]:
json["list"][0].get("rain", {}).get("3h", 0)

0

## Bringing together our chosen items

In [67]:
weather_items = []
# run through every element in the json list. The Item is adictionary. The .get(KEY) method returns the value
for item in json["list"]:
    weather_item = {
        "forecast_time": item.get("dt_txt", None),
        "temperature": item["main"].get("temp", None), # specify the key main --> and then use the get method for this dictionary
        "forecast": item["weather"][0].get("main", None), #specify the key weather --> that is a list - use the first element
        "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
        "wind_speed": item["wind"].get("speed", None),
    }

    weather_items.append(weather_item)

weather_df = pd.DataFrame(weather_items)

weather_df.head()

,forecast_time,temperature,forecast,rain_in_last_3h,wind_speed
0,2024-11-06 15:00:00,8.06,Clouds,0,0.39
1,2024-11-06 18:00:00,8.06,Clouds,0,0.40
2,2024-11-06 21:00:00,7.41,Clouds,0,0.31
3,2024-11-07 00:00:00,6.38,Clouds,0,0.74
4,2024-11-07 03:00:00,5.84,Clouds,0,0.08


The `rain` key in the JSON data is not always present. In the absence of this key, it's assumed that there hasn't been any recent rainfall. Since this key isn't always available, not encountering it during our loop can cause an error. To address this, we'll employ two `.get()` methods to handle the absence of the `rain` key.

> **Remember:** The second parameter of `.get()` serves as a fallback value, which is inserted into our dictionary if the key we're looking for doesn't exist.

The first `.get()` method checks for the existence of the `rain` key. If present, it will use the value associated with that key. If the `rain` key is missing, an empty dictionary will be used, preventing the code from breaking. The second `.get()` method checks for the `3h` key. If present, it will use its value. Otherwise, a default value of `0` will be used.

# Creating a function for multiple cities

In [64]:
import pandas as pd
from datetime import datetime
from pytz import timezone

load_dotenv()

def fetch_weather_data(cities):
  # Let's ensure the timezone is our local timezone
  berlin_timezone = timezone('Europe/Berlin')
  API_key = os.getenv('weather_api_key')
  weather_items = []

  for city in cities:

    country = "DE"

    # Geocoder api to get the latitude and longitude
    # This information should be from your SQL table, but we don't have that here
    geo_url = (f"http://api.openweathermap.org/geo/1.0/direct?q={city},{country}&limit=5&appid={API_key}")
    geo_response = requests.get(geo_url)
    geo_json = geo_response.json()
    latitude = geo_json[0]["lat"]
    longitude = geo_json[0]["lon"]

    # weather api
    weather_url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
    weather_response = requests.get(weather_url)
    weather_json = weather_response.json()

    # Added the time retrieved so we know when the forecast was made
    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in weather_json["list"]:
        weather_item = {
            # Added the city name, so the information is clear when looking at multiple cities
            "city": city,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "forecast": item["weather"][0].get("main", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)

  return weather_df

In [65]:
fetch_weather_data(["Berlin", "Hamburg", "Munich"])

,city,forecast_time,temperature,forecast,rain_in_last_3h,wind_speed,data_retrieved_at
0,Berlin,2024-11-06 18:00:00,7.17,Clouds,0.0,0.40,2024-11-06 16:07:03
1,Berlin,2024-11-06 21:00:00,6.97,Clouds,0.0,0.31,2024-11-06 16:07:03
2,Berlin,2024-11-07 00:00:00,6.38,Clouds,0.0,0.74,2024-11-06 16:07:03
3,Berlin,2024-11-07 03:00:00,5.84,Clouds,0.0,0.08,2024-11-06 16:07:03
4,Berlin,2024-11-07 06:00:00,5.37,Clouds,0.0,1.37,2024-11-06 16:07:03
...,...,...,...,...,...,...,...
115,Munich,2024-11-11 03:00:00,3.95,Clear,0.0,1.09,2024-11-06 16:07:04
116,Munich,2024-11-11 06:00:00,3.50,Clear,0.0,1.32,2024-11-06 16:07:04
117,Munich,2024-11-11 09:00:00,7.74,Clear,0.0,1.13,2024-11-06 16:07:04
118,Munich,2024-11-11 12:00:00,11.12,Clear,0.0,1.51,2024-11-06 16:07:04


# Making our function work with our SQL cities data
This section delves into a more advanced aspect of the project, which may be beyond the point many of you got to. However, it serves as an illustrative example of how to structure your work effectively. It demonstrates how we can compartmentalise various aspects of our data processing pipeline into distinct functions, enhancing code readability and facilitating debugging.
> This part will only work locally, not on colab. You must also create your weather table in SQL first.

In [24]:
CREATE TABLE weather (
    city_id INT UNSIGNED NOT NULL,           -- Matches City_ID type
    forecast_time DATETIME NOT NULL,
    temperature FLOAT,
    forecast VARCHAR(50),
    rain_in_last_3h FLOAT DEFAULT 0,
    wind_speed FLOAT,
    data_retrieved_at DATETIME NOT NULL,
    PRIMARY KEY (city_id, forecast_time),
    FOREIGN KEY (city_id) REFERENCES cities(City_ID) -- Correct reference
);


SyntaxError: '(' was never closed (163804508.py, line 1)

In [85]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from dotenv import load_dotenv
import os


load_dotenv()

def retrieve_and_send_data():
  connection_string = create_connection_string()
  cities_df = fetch_cities_data(connection_string)
  weather_df = fetch_weather_data(cities_df)
  store_weather_data(weather_df, connection_string)
  return "Data has been updated"

def create_connection_string():
  schema = "gans"
  host = "127.0.0.1"
  user = "root"
  password = os.getenv('sql_password')
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

def fetch_weather_data(cities_df):
    berlin_timezone = timezone('Europe/Berlin')
    API_key = os.getenv('weather_api_key')
    weather_items = []  # This will store weather data for all cities.

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_ID"]  # Ensure city_id comes from cities_df.

        # Make API call for the current city
        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        # Iterate through weather forecast items
        for item in weather_data["list"]:
            weather_item = {
                "city_id": city_id,  # Use city_id from cities_df
                "forecast_time": item.get("dt_txt"),
                "temperature": item["main"].get("temp"),
                "forecast": item["weather"][0].get("main"),
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
                "wind_speed": item["wind"].get("speed"),
                "data_retrieved_at": retrieval_time
            }
            weather_items.append(weather_item)

    # Convert the collected weather data into a DataFrame
    weather_df = pd.DataFrame(weather_items)

    # Ensure datetime columns are correctly typed
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

    return weather_df

def store_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                  if_exists='append',
                    con=connection_string,
                    index=False)


In [89]:
weather_df.head()

,city_id,forecast_time,temperature,forecast,rain_in_last_3h,wind_speed,data_retrieved_at
0,1,2024-11-06 18:00:00,6.54,Clouds,0.0,0.40,2024-11-06 17:16:55
1,1,2024-11-06 21:00:00,6.65,Clouds,0.0,0.31,2024-11-06 17:16:55
2,1,2024-11-07 00:00:00,6.38,Clouds,0.0,0.74,2024-11-06 17:16:55
3,1,2024-11-07 03:00:00,5.84,Clouds,0.0,0.08,2024-11-06 17:16:55
4,1,2024-11-07 06:00:00,5.37,Clouds,0.0,1.37,2024-11-06 17:16:55


120

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 32, 37]
})
for index, city in df.iterrows():
    print(f"Row {index} data:")
    print(f"Name: {city['Name']}, Age: {city['Age']}")

Row 0 data:
Name: Alice, Age: 25
Row 1 data:
Name: Bob, Age: 32
Row 2 data:
Name: Charlie, Age: 37


: 

In [ ]:
df

,Name,Age
0,Alice,25
1,Bob,32
2,Charlie,37


: 

## Another option without using `.iterrows()`

In [61]:
import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from dotenv import load_dotenv
import os
import import_ipynb


def retreiving_and_sending_data():
  connection_string = connection()
  cities_df = get_cities_data(connection_string)
  weather_df = get_weather_data(cities_df)
  send_weather_data(weather_df, connection_string)
  return "Data has been updated"

def connection():
  schema = "gans"
  host = "127.0.0.1"
  user = "root"
  password = "YOUR_PASSWORD_HERE"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def get_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

def get_weather_data(cities_df):
  berlin_timezone = timezone('Europe/Berlin')
  API_key = 'YOUR_API_HERE'
  weather_items = []

  for city in cities_df["city"]:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    # As we are now using the data from our relational database
    # the city should reflect the city_id and not the city name
    city_id = cities_df.loc[cities_df["city"] == city, "city_id"].values[0]

    retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for item in json["list"]:
        weather_item = {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "temperature": item["main"].get("temp", None),
            "forecast": item["weather"][0].get("main", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "data_retrieved_at": retrieval_time
        }

        weather_items.append(weather_item)

  weather_df = pd.DataFrame(weather_items)
  weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
  weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

  return weather_df

def send_weather_data(weather_df, connection_string):
  weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [63]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = os.getenv('sql_password')
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

send_weather_data(weather_df, connection_string)

